# Імпорт Бібліотек Йоооооу

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import warnings
warnings.filterwarnings("ignore")


# Генеруємо синтетичний часовий ряд (імітація коливань цін)

In [ ]:
np.random.seed(42)
days = 500
trend = np.linspace(50, 100, days)
seasonal = 10 * np.sin(np.linspace(0, 20 * np.pi, days))
noise = np.random.normal(0, 2, days)

price = trend + seasonal + noise

plt.figure(figsize=(12, 5))
plt.plot(price, label="Synthetic Price")
plt.title("Synthetic Time Series (Price)")
plt.xlabel("Time Step")
plt.ylabel("Price")
plt.legend()
plt.grid()
plt.show()


# Скейлинг
scaler = MinMaxScaler()
price_scaled = scaler.fit_transform(price.reshape(-1, 1))

# Функція для створення послідовностей
def create_sequences(data, window_size):
    sequences = []
    targets = []
    for i in range(len(data) - window_size):
        seq = data[i:i+window_size]
        label = data[i+window_size]
        sequences.append(seq)
        targets.append(label)
    return np.array(sequences), np.array(targets)

window_size = 20
X, y = create_sequences(price_scaled, window_size)

# Train / Test split
split = int(len(X) * 0.8)
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TimeSeriesDataset(X_train_tensor, y_train_tensor)
test_dataset = TimeSeriesDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
